# DATA2001 2021 Assignment
*An analysis of neighbourhood fire risk and median income & rent.*



In [9]:
import pandas as pd
import geopandas as gpd
import os
import numpy as np

In [ ]:
#For troubleshooting in the case your data isn't loading.
#os.chdir('M:\\Jupyter Notebooks\\data2001_project')
os.getcwd()

## Task 1: Data Integration and Database Generation

### Loading the datasets

In [13]:
stat_areas_df = pd.read_csv('./data/StatisticalAreas.csv')
stat_areas_df.head()

,area_id,area_name,parent_area_id
0,1,New South Wales,0
1,10,Greater Sydney,1
2,11,Rest of NSW,1
3,2,Victoria,0
4,20,Greater Melbourne,2


In [15]:
nbhd_df = pd.read_csv('./data/Neighbourhoods.csv')
nbhd_df.head()

,area_id,area_name,land_area,population,number_of_dwellings,number_of_businesses,median_annual_household_income,avg_monthly_rent
0,102011028,Avoca Beach - Copacabana,643.8,7590,2325,738.0,46996.0,1906.0
1,102011029,Box Head - MacMasters Beach,3208.6,10986,3847,907.0,42621.0,1682.0
2,102011030,Calga - Kulnura,76795.1,4841,1575,1102.0,42105.0,1182.0
3,102011031,Erina - Green Point,3379.3,14237,4450,1666.0,43481.0,1595.0
4,102011032,Gosford - Springfield,1691.2,19385,6373,2126.0,45972.0,1382.0


In [16]:
busi_stat_df = pd.read_csv('./data/BusinessStats.csv')
busi_stat_df.head()

,area_id,area_name,number_of_businesses,accommodation_and_food_services,retail_trade,agriculture_forestry_and_fishing,health_care_and_social_assistance,public_administration_and_safety,transport_postal_and_warehousing
0,101021007,Braidwood,629,26,27,280,11,0,35
1,101021008,Karabar,326,7,10,8,11,0,43
2,101021009,Queanbeyan,724,52,47,11,56,3,77
3,101021010,Queanbeyan - East,580,16,23,4,12,0,57
4,101021011,Queanbeyan Region,1642,39,63,292,34,7,81


In [18]:
rfs_df = gpd.read_file('./data/RFSNSW_BFPL/RFSNSW_BFPL.shp')
rfs_df.head()

,CATEGORY,SHAPE_LENG,SHAPE_AREA,geometry
0,1,0.000017,5.392400e-12,POINT (149.11319 -33.05824)
1,1,0.000178,1.140005e-09,POINT (152.27536 -29.68316)
2,1,0.000890,4.950178e-08,POINT (152.14244 -29.68266)
3,1,0.000442,8.094091e-09,POINT (152.27579 -29.68259)
4,1,0.000890,4.950155e-08,POINT (151.99619 -29.68131)


## Task 2: Fire Risk Analysis

### Fire Risk Score

$$fire_risk = S(z(population_density)+z(dwelling_&_business_density)+z(bfpl_density)−z(assistive_service_density))$$

In [21]:
#Z-score
def z(x, avg, sd):
    return((x-avg)/sd)

#Sigmoidal function. Did not use native exponential because fails for large negative values.
def sigmoid(x):
    return(1/(1+np.exp(-x)))

#Fire risk score formula.
def fire_risk(pop_d, dwell_bus_d, bfpl_d, ass_serv_d):
    fire_risk_score = sigmoid(z(pop_d)+z(dwell_bus_d)+z(bfpl_d)-z(ass_serv_d))
    return fire_risk_score